# Hustle Data
- What groups do we want to keep, which don't we want?
- Which organizers are texting on hustle?

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
ooc_actions = pd.read_csv(r'C:\Users\OOC User\Documents\OOC\Data\Hustle\Ohio Organizing Account\actions_02232022.csv', dtype='str')
ooc_messages = pd.read_csv(r'C:\Users\OOC User\Documents\OOC\Data\Hustle\Ohio Organizing Account\messages_02232022.csv', dtype='str')
ooc_leads = pd.read_csv(r'C:\Users\OOC User\Documents\OOC\Data\Hustle\Ohio Organizing Account\leads_02232022.csv', dtype='str')
ooc_leads = ooc_leads.loc[:, :'organization_name']

cca_actions = pd.read_csv(r'C:\Users\OOC User\Documents\OOC\Data\Hustle\CCA Account\cca_actions.csv', dtype='str')
cca_messages = pd.read_csv(r'C:\Users\OOC User\Documents\OOC\Data\Hustle\CCA Account\cca_messages.csv', dtype='str')
cca_leads = pd.read_csv(r'C:\Users\OOC User\Documents\OOC\Data\Hustle\CCA Account\cca_leads.csv', dtype='str')
cca_leads = cca_leads.loc[:, :'organization_name']

In [3]:
cca_messages.columns

Index(['message_created_at', 'message_type', 'message_direction',
       'message_text', 'first_name', 'last_name', 'phone_number', 'email',
       'agent_name', 'agent_phone_number', 'agent_email', 'goal_id',
       'goal_name', 'group_id', 'group_name', 'group_integrations',
       'organization_id', 'organization_name', 'custom_field_zip',
       'custom_field_city', 'custom_field_state', 'custom_field_source',
       'custom_field_timestamp', 'custom_field_name', 'custom_field_path',
       'custom_field_constituent', 'custom_field_assessment',
       'custom_field_sq1', 'custom_field_sq2', 'custom_field_sq3',
       'custom_field_sq4', 'custom_field_poll', 'custom_field_polladdress',
       'custom_field_pollname', 'custom_field_pollsite', 'custom_field_pdi_id',
       'custom_field_preferred_language', 'custom_field_provider_type',
       'custom_field_parentguardian_type', 'custom_field_actionbuilderid',
       'custom_field_middle_name', 'custom_field_is_organization',
       '

In [4]:
#mapping ohio general group to unemployment and childcare
cca_messages_join = cca_messages[['phone_number', 'agent_name', 'agent_phone_number', 'agent_email']].drop_duplicates()
cca_leads = pd.merge(left=cca_leads, left_on='phone_number', right=cca_messages_join, right_on='phone_number', how='left', indicator=True)

cca_leads.loc[cca_leads['agent_name']=='Janiah', 'group_name'] = 'Unemployment'
unemployment = cca_leads.loc[cca_leads['group_name']=='Unemployment', :]
unemployment = unemployment[['phone_number', 'first_name', 'last_name', 'email', 'agent_name', 'agent_phone_number', 'agent_email']]
unemployment = (unemployment.rename(
    {'phone_number':'Phone number', 
     'first_name':'First name', 
     'last_name':'Last name', 
     'email':'Email', 
     'agent_name':'Agent first name', 
     'agent_phone_number':'Agent phone number', 
     'agent_email':'Agent email'}, 
    axis=1))


cca_leads.loc[cca_leads['agent_name']=='Tami', 'group_name'] = 'Child Care'
child_care = cca_leads.loc[cca_leads['group_name']=='Child Care', :]
child_care = child_care[['phone_number', 'first_name', 'last_name', 'email', 'agent_name', 'agent_phone_number', 'agent_email']]
child_care = (child_care.rename(
    {'phone_number':'Phone number', 
     'first_name':'First name', 
     'last_name':'Last name', 
     'email':'Email', 
     'agent_name':'Agent first name', 
     'agent_phone_number':'Agent phone number', 
     'agent_email':'Agent email'}, 
    axis=1))


cca_leads = cca_leads.drop(['agent_name', '_merge', 'agent_phone_number'], axis=1)

In [5]:
# export unemployment and childcare lists for temporary OOC hustle until cca hustle is 10dlc compliant
child_care.to_csv('child care.csv', index=False)
unemployment.to_csv('unemployment.csv', index=False)

In [6]:
actions = pd.concat([ooc_actions, cca_actions])
messages = pd.concat([ooc_messages, cca_messages])
leads = pd.concat([ooc_leads, cca_leads])

#### Actions - All rsvps that people we text make (accepts and declines)
#### Messages - All texts sent
#### Leads - Contacts, by group

# What groups do we want to keep, which don't we want?

## Top groups in order of how recently someone new has been added

In [7]:
#format date
leads['created_at'] = leads['created_at'].astype(str)
leads['created_at'] = leads['created_at'].str[:10]
leads['created_at'] = pd.to_datetime(leads['created_at'], infer_datetime_format=True)
# leads['created_at']

(leads.groupby('group_name')['created_at'].agg(['max', 'size'])
     .sort_values(by='max', ascending=False)
     .rename({'max':'last_new_member', 'size':'People in group'}, axis=1)
     # .head(10)
)

,last_new_member,People in group
group_name,,
OSA Cleveland State,2022-02-22,145
OSA University of Akron,2022-02-17,60
Unemployment,2022-02-01,460
OSA CWRU,2022-02-01,36
Ohio General,2022-02-01,734
Child Care,2021-08-27,360
SURJ Ohio,2020-12-16,5211
SURJ NEO,2020-09-22,3003
OSA Cleveland,2020-05-31,301


## Top groups in order of the last time someone in the group performed an action
    

In [8]:
#format date
actions['action_created_at'] = actions['action_created_at'].astype(str)
actions['action_created_at'] = actions['action_created_at'].str[:10]
actions['action_created_at'] = pd.to_datetime(actions['action_created_at'], infer_datetime_format=True)
# actions['created_at']

(actions.groupby('group_name')['action_created_at'].agg(['max', 'size'])
     .sort_values(by='max', ascending=False)
     .rename({'max':'last_action', 'size':'total actions'}, axis=1)
     # .head(10)
)

,last_action,total actions
group_name,,
OSA Cleveland State,2022-02-23,26
OSA University of Akron,2022-02-22,14
Ohio General,2022-02-09,216
OSA CWRU,2022-02-09,6
SURJ Ohio,2021-05-24,974
SURJ NEO,2021-04-06,828
Cuyahoga County Jail Reform,2020-07-24,1352
OSA Cleveland,2020-06-02,60
GOTV - Volunteer,2018-12-07,663


## Top groups in order of the last time someone in the group was texted

In [9]:
#format date
messages['message_created_at'] = messages['message_created_at'].astype(str)
messages['message_created_at'] = messages['message_created_at'].str[:10]
messages['message_created_at'] = pd.to_datetime(messages['message_created_at'], infer_datetime_format=True)
# messages['created_at']

(messages.groupby('group_name')['message_created_at'].agg(['max', 'size'])
     .sort_values(by='max', ascending=False)
     .rename({'max':'last_message', 'size':'total_texts'}, axis=1)
     # .head(10)
)


,last_message,total_texts
group_name,,
Ohio General,2022-02-26,4785
OSA University of Akron,2022-02-23,118
OSA Cleveland State,2022-02-23,778
OSA CWRU,2022-02-16,100
Cuyahoga County Jail Reform,2022-01-01,31844
SURJ Ohio,2021-12-31,27444
SURJ NEO,2021-05-30,15619
Issue 44,2020-11-11,3076
OSA Cleveland,2020-06-04,557


# Who's texting on Hustle?

## Last people to send a text to any group

In [10]:
(messages.groupby('agent_name')['message_created_at'].agg(['max', 'size'])
     .sort_values(by='max', ascending=False)
     .rename({'max':'last_message', 'size':'total_messages_sent'}, axis=1)
     .head(15)
)

,last_message,total_messages_sent
agent_name,,
Janiah,2022-02-26,676
Rachael,2022-02-23,6420
Tami,2022-02-15,2460
Lane,2022-02-10,317
Samira,2022-01-01,3632
Anna,2021-12-31,2787
Julie,2021-06-29,1293
Chris,2021-06-26,2479
Lucie,2021-05-30,951
